# Ingest data from data source to S3 bucket

***

## Libraries

In [2]:
import sagemaker
import boto3
from datetime import datetime
from io import StringIO
import pandas as pd
from tqdm import tqdm
import seaborn as sns

## Variables

In [3]:
sess = sagemaker.Session() #Intialzie session

bucket = sess.default_bucket()
RD_Bucket = 'sdpd-bucket' #Raw Data
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3 = boto3.client('s3') # Create an S3 client
s3_resource = boto3.resource('s3') # Create an S3 resource

print('Bucket - > ',bucket)
print('Role - > ',role)
print('Region - > ',region)
print('S3 - > ',s3)
print('S3 Resource - > ',s3_resource)

Bucket - >  sagemaker-us-east-1-109424293498
Role - >  arn:aws:iam::109424293498:role/LabRole
Region - >  us-east-1
S3 - >  <botocore.client.S3 object at 0x7fc26cadcad0>
S3 Resource - >  s3.ServiceResource()


## Prepare landing zone

In [4]:
# Function to verify if bucket exist, if not create
def verify_create_bucket(bucket_name):
    response = s3.list_buckets()
    for bucket in response['Buckets']:
        if bucket['Name'] == bucket_name:
            print(f"The {bucket_name} bucket exists.")
            break
    else:
        print(f"The {bucket_name} bucket does not exist, creating")
        !aws s3 mb s3://{RD_Bucket}/

In [5]:
verify_create_bucket(RD_Bucket)

The sdpd-bucket bucket exists.


## Download Files

***

### Load calls

In [6]:
#function to download yearly data
def Get_Data(year):
    url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
    df = pd.read_csv(url)
    return df

In [49]:
# List of years since 2018
Years = list(range(2021,datetime.now().year+1))
Years

[2021, 2022, 2023]

In [50]:
sdpd_calls = pd.DataFrame()
#iterate over years
for year in tqdm(Years):
    year_df = Get_Data(year)
    year_df['Year'] = year
    sdpd_calls = pd.concat((sdpd_calls, year_df))
    
    #csv_buffer = StringIO()
    #year_df.to_csv(csv_buffer, index=False)
    #file = 'SDPD_Calls_' + str(year) + '.csv'
    #s3_resource.Object(RD_Bucket,file).put(Body=csv_buffer.getvalue())
    #print(file,'loaded in',RD_Bucket,'bucket')

100%|██████████| 3/3 [00:07<00:00,  2.35s/it]


In [51]:
#Verify Shape
sdpd_calls.shape

(1168014, 15)

In [52]:
sdpd_calls = sdpd_calls.drop(['address_dir_intersecting', 'address_sfx_intersecting'], axis=1)

***

In [53]:
type_url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
dipo_url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
ripa_stops_url = "https://seshat.datasd.org/pd/ripa_stops_datasd.csv"
ripa_stops_dic = "https://seshat.datasd.org/pd/ripa_stops_dictionary_datasd.csv"

In [54]:
type_df = download_file(type_url)
dispo_df = download_file(dipo_url)
ripa_stops_df = download_file(ripa_stops_url)
ripa_stops_dic_df = download_file(ripa_stops_dic)


***

## Data Quality Report

In [55]:
def Data_Quality_Report(df):

    #Initial table
    freqDF = pd.DataFrame(columns=['Feature',
                                   'Mode',
                                   'Mode Freq.',
                                   'Mode %',
                                   '2nd Mode',
                                   '2nd Mode Freq.',
                                   '2nd Mode %'])
    for col in tqdm(df.columns):
        try:
            #print(col)
            freq = df[col].value_counts()
            freqdf = freq.to_frame()
            fRow = freqdf.iloc[0]
            #try:
            secRow = freqdf.iloc[1]
            #except:
            #secRow = 0
            fPrct = fRow[0] / len(df[col])
            #try:
            secPrct = secRow[0] / len(df[col])
            #except:
                #secPrct = 0
            try:
                mode1 = int(fRow.name)
            except:
                mode1 = fRow.name
            try:
                mode2 = int(secRow.name)
            except:
                try:
                    mode2 = secRow.name
                except:
                    mode2 = 0
            freqDF = freqDF.append({'Feature':col,
                                    'Mode':mode1,
                                    'Mode Freq.':fRow[0],
                                    'Mode %':fPrct,\
                                    '2nd Mode':mode2,
                                    '2nd Mode Freq.':secRow[0],
                                    '2nd Mode %':secPrct},
                                    ignore_index=True)
        except:
            pass

    freqDF = freqDF.set_index('Feature')

    #Nulls, Counts, Cardinality
    NUllFeatures = round(df.isnull().sum() / df.shape[0],4)\
          .sort_values(ascending=False)
    Count = df.count()
    uni = df.nunique()

    #Formating
    NUllFeatures.to_frame(name="% Miss.")
    Count.to_frame(name="Count")
    uni.to_frame()
    result = pd.concat([Count, NUllFeatures,uni], axis=1)
    result.columns =["Count","% Miss.","Card."]
    result = pd.concat([result, freqDF], axis=1)
    result = result.style.format({'% Miss.': "{:.1%}",
                         'Mode %': "{:.0%}",
                         '2nd Mode %': "{:.0%}",
                         'Count': "{:,}",
                         'Card.': "{:,}",
                         'Mode Freq.': "{:,}",
                        '2nd Mode Freq.': "{:,}"})
    return result

In [56]:
DQR_Calls = Data_Quality_Report(sdpd_calls)
DQR_Calls

100%|██████████| 13/13 [00:02<00:00,  5.37it/s]


,Count,% Miss.,Card.,Mode,Mode Freq.,Mode %,2nd Mode,2nd Mode Freq.,2nd Mode %
incident_num,"1,168,014",0.0%,"1,168,014",E21010000001,1,0%,E22050043191,1,0%
date_time,"1,168,014",0.0%,"1,157,922",2023-02-23 10:54:45,3,0%,2023-01-27 18:43:23,3,0%
day_of_week,"1,168,014",0.0%,7,5,"175,015",15%,6,"173,878",15%
address_number_primary,"1,168,014",0.0%,262,0,"212,027",18%,1400,"24,613",2%
address_dir_primary,"55,064",95.3%,9,W,"19,522",2%,S,"17,418",1%
address_road_primary,"1,167,818",0.0%,"12,704",IMPERIAL,"21,385",2%,05TH,"18,939",2%
address_sfx_primary,"1,067,996",8.6%,42,ST,"348,278",30%,AVE,"275,917",24%
address_road_intersecting,"196,743",83.2%,"6,234",MARKET,"5,374",0%,UNIVERSITY,"5,072",0%
call_type,"1,166,864",0.1%,251,415,"102,096",9%,SELENF,"68,144",6%
disposition,"1,164,029",0.3%,19,K,"608,343",52%,O,"173,216",15%


In [57]:
def download_file(url):
    df = pd.read_csv(url, low_memory=False)
    return df

In [58]:
DQR_Stops = Data_Quality_Report(ripa_stops_df)
DQR_Stops

100%|██████████| 29/29 [00:00<00:00, 44.71it/s]


,Count,% Miss.,Card.,Mode,Mode Freq.,Mode %,2nd Mode,2nd Mode Freq.,2nd Mode %
stop_id,"653,601",0.0%,"574,368",174011,52,0%,184085,48,0%
ori,"653,601",0.0%,1,nan,nan,nan%,nan,nan,nan%
agency,"653,601",0.0%,1,nan,nan,nan%,nan,nan,nan%
exp_years,"653,601",0.0%,41,1,"256,905",39%,3,"49,912",8%
date_stop,"653,601",0.0%,"1,645",2020-02-12,799,0%,2019-05-23,793,0%
time_stop,"653,601",0.0%,"83,477",16:00:00,"1,772",0%,10:00:00,"1,492",0%
stopduration,"653,601",0.0%,398,10,"157,652",24%,15,"77,367",12%
stop_in_response_to_cfs,"653,601",0.0%,2,0,"584,869",89%,1,"68,732",11%
officer_assignment_key,"653,601",0.0%,10,1,"610,968",93%,10,"20,167",3%
assignment,"653,601",0.0%,10,"Patrol, traffic enforcement, field operations","610,968",93%,Other,"20,167",3%


***

## Save Date to Buckets

In [59]:
def move_2_bucket(bucket,subfolder_name,file_name,df):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_resource.Object(bucket,
                       subfolder_name + file_name).put(Body=csv_buffer.getvalue())
    print(file_name,'loaded in',bucket,'bucket')
    

In [60]:
move_2_bucket(RD_Bucket,'Call_Data/','SDPD_Calls.csv',sdpd_calls)
move_2_bucket(RD_Bucket,'Type_Data/','SDPD_Type.csv',type_df)
move_2_bucket(RD_Bucket,'Dispo_Data/','SDPD_Dispo.csv',dispo_df)
move_2_bucket(RD_Bucket,'Stops_Data/','SDPD_Stops.csv',ripa_stops_df)
move_2_bucket(RD_Bucket,'Stops_Dic_Data/','SDPD_Stops_Dic.csv',ripa_stops_dic_df)

SDPD_Calls.csv loaded in sdpd-bucket bucket
SDPD_Type.csv loaded in sdpd-bucket bucket
SDPD_Dispo.csv loaded in sdpd-bucket bucket
SDPD_Stops.csv loaded in sdpd-bucket bucket
SDPD_Stops_Dic.csv loaded in sdpd-bucket bucket


<div class="alert alert-block alert-success">
<b>Data:</b> Load in S3 Buckets
</div>

***

## Aggregate Police Acticity by Hour

In [72]:
def events_hour_timeseries(df,date_field):
    df[date_field]=pd.to_datetime(df[date_field])
    df_group = df.groupby([
        pd.Grouper(key=date_field,freq='H')
    ]).size().reset_index(name='count')
    
    return df_group

In [72]:
def events_hour_timeseries(df,date_field):
    df[date_field]=pd.to_datetime(df[date_field])
    df_group = df.groupby([
        pd.Grouper(key=date_field,freq='H')
    ]).size().reset_index(name='count')
    
    return df_group

In [74]:
time_series_prior_series = events_hour_timeseries(sdpd_calls,'date_time')

In [74]:
time_series_prior_series = events_hour_timeseries(sdpd_calls,'date_time')

In [69]:
time_series

,date_time,priority,count
0,2021-01-01 00:00:00,0,6
1,2021-01-01 00:00:00,1,44
2,2021-01-01 00:00:00,2,27
3,2021-01-01 00:00:00,3,29
4,2021-01-01 00:00:00,4,4
...,...,...,...
101076,2023-03-15 22:00:00,3,4
101077,2023-03-15 22:00:00,4,2
101078,2023-03-15 23:00:00,1,2
101079,2023-03-15 23:00:00,2,8


***

## Visualize

In [ ]:
sns.lineplot(x="date_time",
             y="count",
             #hue="region",
             style="event",
             data=time_series)